# 나무위키 크롤러 생성

## 1. 나무위키 구조 정의

<p>
    나무위키 컨탠츠 구조<br>
    소제목 : h2(class:wiki-heading) -> span(id=소제목)<br>
    내용 : div(class:wiki-heading-content) -> div(class:wiki-paragraph)
</p>


In [ ]:
#테스트
import requests
from bs4 import BeautifulSoup

req = requests.get('https://namu.wiki/w/녹두로')
bs = BeautifulSoup(req.text, 'html.parser')

targets = bs.find_all("h2", class_="wiki-heading")
for target in targets:
    realtarget = target.next_sibling.get_text()
    print(realtarget)

In [ ]:
#크롤러
import requests
from bs4 import BeautifulSoup

class Namu_Crawler:
    def __init__(self, keyword):
        self.keyword = keyword

    def getPage(self):
        try:
            req = requests.get('https://namu.wiki/w/'+self.keyword)
        except requests.exceptions.RequestException:
            print('request error')
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def crawl(self):
        youtuberstring = []
        bs = self.getPage()
        targets = bs.find_all("h2", class_="wiki-heading")
        for target in targets:
            youtuberstring.append(target.next_sibling.get_text())
        return youtuberstring

crawler = Namu_Crawler('감스트')
strings = "".join(crawler.crawl())
strings



In [49]:
#워드클라우드
# -*- encoding:utf8 -*-

from collections import Counter
import random

from konlpy.tag import Hannanum
import pytagcloud
import webbrowser

r = lambda: random.randint(0,255)
color = lambda: (r(),r(),r())

def get_tags(text, ntags=50, multiplier=10):
    h = Hannanum()
    nouns = h.nouns(text)
    count = Counter(nouns)
    return [{ 'color': color(), 'tag': n, 'size': c*multiplier }\
                for n, c in count.most_common(ntags)]

def draw_cloud(tags, filename, fontname='Korean', size=(800, 600)):
    pytagcloud.create_tag_image(tags, filename, fontname=fontname, size=size)
    webbrowser.open(filename)


tags = get_tags(strings)
print(tags)
draw_cloud(tags,'testcloud.png')



[{'color': (216, 80, 18), 'tag': '방송', 'size': 530}, {'color': (172, 194, 229), 'tag': '말', 'size': 480}, {'color': (214, 149, 229), 'tag': '감스트', 'size': 380}, {'color': (14, 45, 254), 'tag': '게임', 'size': 290}, {'color': (153, 73, 188), 'tag': '것', 'size': 270}, {'color': (145, 142, 53), 'tag': '시청자들', 'size': 200}, {'color': (57, 110, 43), 'tag': '본인', 'size': 200}, {'color': (16, 30, 46), 'tag': '중', 'size': 190}, {'color': (49, 53, 237), 'tag': '때', 'size': 180}, {'color': (218, 133, 242), 'tag': '라', 'size': 140}, {'color': (167, 112, 168), 'tag': '등', 'size': 130}, {'color': (198, 159, 200), 'tag': '이후', 'size': 130}, {'color': (81, 101, 168), 'tag': '수', 'size': 130}, {'color': (83, 7, 37), 'tag': '콘텐츠', 'size': 120}, {'color': (38, 119, 203), 'tag': '리그', 'size': 120}, {'color': (241, 17, 146), 'tag': '컨텐츠', 'size': 120}, {'color': (202, 195, 20), 'tag': '경우', 'size': 120}, {'color': (255, 72, 142), 'tag': '시작', 'size': 110}, {'color': (6, 115, 208), 'tag': '복귀', 'size': 110},

AttributeError: Invalid font name. Should be one of Nobile, Old Standard TT, Cantarell, Reenie Beanie, Cuprum, Molengo, Neucha, Philosopher, Yanone Kaffeesatz, Cardo, Neuton, Inconsolata, Crimson Text, Josefin Sans, Droid Sans, Lobster, IM Fell DW Pica, Vollkorn, Tangerine, Coustard, PT Sans Regular